In [11]:
import json
import logging

SITE = "http://ieda.ust.hk/eng/event_detail.php?type=E&id="

settings = None
try:
    with open("settings.json", "r") as f:
        settings = json.load(f)
except FileNotFoundError:
    logging.error("settings.json not found")

eventID = 802

import urllib.request
contents = urllib.request.urlopen(SITE+str(eventID)).read()

from bs4 import BeautifulSoup
soup = BeautifulSoup(contents, 'html.parser')

In [12]:
title = soup.find(class_='context__subtitle').text.strip()
title

'Joint OM/IE Seminar'

In [13]:
soup.find_all('span', class_='item-label')


[<span class="item-label">Date</span>,
 <span class="item-label">Time</span>,
 <span class="item-label">Venue</span>,
 <span class="item-label">Speaker</span>]

In [14]:
soup.find_all('span', class_='item-value')

for label, value in zip(soup.find_all('span', class_='item-label'), soup.find_all('span', class_='item-value')):
    print(label.text, value.text)

Date 2 February 2024 (Friday)
Time 10:30 am
Venue LSK - Room 4047
Speaker Prof. Burak KAZAZ , Professor


In [15]:
HOMEPAGE = "http://ieda.ust.hk/eng/events.php?catid=6&sid=50"

contents_home = urllib.request.urlopen(HOMEPAGE).read()
soup2 = BeautifulSoup(contents_home, 'html.parser')

In [27]:
int(soup2.find('div', class_='info-block__date active').attrs['data-tab'])

802